<a href="https://colab.research.google.com/github/RonVest92/LMIC-Health-Insights-ToolKit/blob/main/AI_Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌍 LMIC Health Insights Toolkit Project
### Author: Ronald Ddibya

# INTRODUCTION

This notebook demonstrates an end-to-end AI prototype.
It simulates a lightweight, ethical, and deployable AI solution for LMIC health contexts. The project is dependable on the following features:
- Synthetic data generation (no PII)
- Model training & evaluation
- Drift detection
- Fairness checks
- Agent-based insights
- Chatbot prototype
- A/B test simulation

The project simulates LMIC-like health data for a diagnostic aide prototype.
#### Features: demographics, symptoms, environmental proxies.
#### Target: positive_diagnostic (1 = likely positive, 0 = likely negative)

In [2]:
# =========================================
# 1. SETUP
# =========================================
!pip install pandas numpy scikit-learn joblib matplotlib seaborn shap fastapi uvicorn streamlit

import pandas as pd
import numpy as np
import joblib
import json
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support, classification_report, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

SEED = 42
np.random.seed(SEED)

# =========================================
# 2. SYNTHETIC DATA GENERATION
# =========================================
def generate_synthetic(n=8000):
    regions = ["urban_low_resource", "rural_low_resource", "peri_urban"]
    sex = ["female", "male"]
    age = np.clip(np.random.normal(28, 12, n).astype(int), 0, 85)
    sex_col = np.random.choice(sex, size=n, p=[0.52, 0.48])
    region = np.random.choice(regions, size=n, p=[0.35, 0.5, 0.15])
    fever = np.random.binomial(1, 0.55, n)
    headache = np.random.binomial(1, 0.5, n)
    chills = np.random.binomial(1, 0.45, n)
    fatigue = np.random.binomial(1, 0.5, n)
    travel_last_30d = np.random.binomial(1, 0.2, n)
    bednet_use = np.random.binomial(1, 0.6, n)
    rainfall_idx = np.clip(np.random.normal(0.1, 0.9, n), -2, 3)

    logit = (
        -2.0 + 0.9*fever + 0.6*chills + 0.5*headache + 0.4*fatigue
        + 0.5*travel_last_30d - 0.3*bednet_use + 0.25*rainfall_idx + 0.002*age
    )
    region_adj = {"urban_low_resource": 0.2, "rural_low_resource": 0.6, "peri_urban": 0.4}
    for i, r in enumerate(region):
        logit[i] += region_adj[r]
    prob = 1.0 / (1.0 + np.exp(-logit))
    positive = np.random.binomial(1, prob)

    return pd.DataFrame({
        "age": age, "sex": sex_col, "region": region,
        "fever": fever, "headache": headache, "chills": chills,
        "fatigue": fatigue, "travel_last_30d": travel_last_30d,
        "bednet_use": bednet_use, "rainfall_idx": rainfall_idx,
        "positive_diagnostic": positive
    })

df = generate_synthetic()
df.head()

# =========================================
# 3. FEATURE ENGINEERING
# =========================================
NUM_COLS = ["age", "rainfall_idx"]
BIN_COLS = ["fever", "headache", "chills", "fatigue", "travel_last_30d", "bednet_use"]
CAT_COLS = ["sex", "region"]
TARGET_COL = "positive_diagnostic"

def build_preprocessor():
    numeric = StandardScaler()
    categorical = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    return ColumnTransformer([
        ("num", numeric, NUM_COLS),
        ("bin", "passthrough", BIN_COLS),
        ("cat", categorical, CAT_COLS)
    ])

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# =========================================
# 4. MODEL TRAINING & EVALUATION
# =========================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
pipe = Pipeline([
    ("pre", build_preprocessor()),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])
pipe.fit(X_train, y_train)

y_proba = pipe.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= 0.5).astype(int)
auc = roc_auc_score(y_test, y_proba)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary")
print(f"AUC: {auc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
print(classification_report(y_test, y_pred))

# =========================================
# 5. DRIFT DETECTION
# =========================================
# Only use numeric columns to avoid TypeError from categorical data
num_cols = ref = df.iloc[: int(0.7*len(df))].drop(columns=[TARGET_COL]).select_dtypes(include=[np.number]).columns
ref = df.iloc[: int(0.7*len(df))]
new = df.iloc[int(0.7*len(df)) :]

drift_score = mean_squared_error(ref[num_cols].mean(), new[num_cols].mean())
print(f"Drift Score (MSE of means, numeric only): {drift_score:.4f}")

# =========================================
# 6. FAIRNESS CHECKS
# =========================================
def subgroup_metrics(X_df, y_true, y_pred, sensitive_cols):
    rows = []
    for col in sensitive_cols:
        for grp, idx in X_df.groupby(col).groups.items():
            yt = y_true.loc[idx]
            yp = y_pred.loc[idx]
            prec, rec, f1, _ = precision_recall_fscore_support(yt, yp, average="binary", zero_division=0)
            rows.append({"feature": col, "group": grp, "precision": prec, "recall": rec, "f1": f1})
    return pd.DataFrame(rows)

SENSITIVE_COLS = ["sex", "region"]
fairness_df = subgroup_metrics(X.reset_index(drop=True), y.reset_index(drop=True), pd.Series(pipe.predict(X)), SENSITIVE_COLS)
fairness_df

# =========================================
# 7. AGENT-BASED INSIGHTS
# =========================================
def weekly_insights(df):
    n = len(df)
    prev = df.iloc[:-7] if n > 14 else df.iloc[: n//2]
    curr = df.iloc[-7:] if n > 14 else df.iloc[n//2 :]
    msg = []
    msg.append(f"Weekly insights (n={len(curr)} vs prior n={len(prev)}):")
    for col in ["fever","chills","headache","fatigue","positive_diagnostic"]:
        msg.append(f"- Change in mean {col}: {curr[col].mean():.3f} vs {prev[col].mean():.3f}")
    return "\n".join(msg)

print(weekly_insights(df))

# =========================================
# 8. CHATBOT PROTOTYPE
# =========================================
RESPONSES = {
    "fever": "Fever can indicate multiple conditions. Seek testing if severe.",
    "chills": "Chills often co-occur with fever. Stay hydrated.",
    "bednet": "Bed nets reduce malaria risk. Use nightly.",
    "help": "I can provide general guidance. For emergencies, contact a health worker."
}

def respond(user_text):
    t = user_text.lower()
    for k, v in RESPONSES.items():
        if k in t:
            return v + " (Prototype guidance, not medical advice.)"
    return "Please share your main symptom and any recent travel."

print(respond("I have fever and chills"))

# =========================================
# 9. A/B TEST SIMULATION (continued)
# =========================================
from scipy import stats

def simulate_ab(n=1000, conv_a=0.18, conv_b=0.22, seed=SEED):
    rng = np.random.default_rng(seed)
    a = rng.binomial(1, conv_a, n)
    b = rng.binomial(1, conv_b, n)
    # two-proportion z-test
    p1, p2 = a.mean(), b.mean()                  #... Calculate conversion rates
    p_pool = (a.sum() + b.sum()) / (2*n)         # Pooled proportion
    se = np.sqrt(p_pool * (1 - p_pool) * (2/n))  # Standard error
    z = (p2 - p1) / se                           # Z-score and p-value
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    return {
        "conv_a": round(p1, 4),
        "conv_b": round(p2, 4),
        "z_score": round(z, 4),
        "p_value": round(p_value, 4),
        "significant": p_value < 0.05
    }

ab_results = simulate_ab()                      # Run the simulation
print("A/B Test Results:", ab_results)

# =========================================
# 10. WRAP-UP
# =========================================
print("\n--- PROJECT SUMMARY ---")
print(f"Model AUC: {auc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
print(f"Drift Score: {drift_score:.4f}")
print("Fairness metrics sample:\n", fairness_df.head())
print("Weekly Insights:\n", weekly_insights(df))
print("Chatbot sample response:", respond("I have fever and chills"))
print("A/B Test:", ab_results)


AUC: 0.6828, Precision: 0.5633, Recall: 0.6076, F1: 0.5846
              precision    recall  f1-score   support

           0       0.69      0.64      0.66       912
           1       0.56      0.61      0.58       688

    accuracy                           0.63      1600
   macro avg       0.62      0.63      0.62      1600
weighted avg       0.63      0.63      0.63      1600

Drift Score (MSE of means, numeric only): 0.0062
Weekly insights (n=7 vs prior n=7993):
- Change in mean fever: 0.857 vs 0.552
- Change in mean chills: 0.429 vs 0.447
- Change in mean headache: 0.286 vs 0.496
- Change in mean fatigue: 0.571 vs 0.508
- Change in mean positive_diagnostic: 0.571 vs 0.430
Fever can indicate multiple conditions. Seek testing if severe. (Prototype guidance, not medical advice.)
A/B Test Results: {'conv_a': np.float64(0.18), 'conv_b': np.float64(0.234), 'z_score': np.float64(2.9803), 'p_value': np.float64(0.0029), 'significant': np.True_}

--- PROJECT SUMMARY ---
Model AUC: 0.6828